In [ ]:
%env CUDA_VISIBLE_DEVICES=5

In [2]:
import numpy as np
import cupy as cp

In [3]:
xp = cp

X = 1500
Y = 1500
Z = 20
K = 3
data = np.random.rand(X,Y,Z).astype('float32')

# numpy

In [4]:
def vectors_comparison(vector1, vector2):
    res = xp.multiply(vector1, vector2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def vectors_comparison1(matrix1, matrix2):
    return (xp.power(matrix1 * matrix2, 2)).sum(axis=2)

def compare_matrices(a_pad, data, xk, yk, K, comp_function):
    comparison = comp_function(data, a_pad[xk:xk+data.shape[0], K+yk:K+yk+data.shape[1], :])
    res = xp.pad(
        comparison[:data.shape[0]-xk, max(0, -yk):min(data.shape[1], data.shape[1]-yk)], 
        ((xk, 0), (max(0, yk), -min(0, yk))), 
        constant_values=xp.nan
    )
    return comparison, res

def get_comparison(data, X, Y, Z, K, comp_function):
    K = K + 1
    a_pad = xp.pad(data, ((0, K), (K, K), (0, 0)), constant_values=xp.nan)
    result = []    
    for xk in range(0, K):
        for yk in range(0, K):
            if (xk!=0) or (yk!=0):   
                result.extend(compare_matrices(a_pad, data, xk, yk, K, comp_function))
                if (yk!=0) and (xk!=0):
                    result.extend(compare_matrices(a_pad, data, xk, -yk, K, comp_function))
    cp.cuda.stream.get_current_stream().synchronize()
    return result

In [5]:
%%time
data_n = cp.asarray(data)

CPU times: user 94.8 ms, sys: 353 ms, total: 447 ms
Wall time: 3.72 s


In [6]:
%%time
res2 = get_comparison(data_n, X, Y, Z, K, vectors_comparison)

CPU times: user 531 ms, sys: 34.6 ms, total: 566 ms
Wall time: 3.33 s


In [7]:
%%time
res2 = get_comparison(data_n, X, Y, Z, K, vectors_comparison)

CPU times: user 72.8 ms, sys: 17.8 ms, total: 90.6 ms
Wall time: 89.6 ms


In [8]:
%%time
res2 = get_comparison(data_n, X, Y, Z, K, vectors_comparison)

CPU times: user 75.4 ms, sys: 15.3 ms, total: 90.7 ms
Wall time: 89.6 ms


In [9]:
%%timeit
res2 = get_comparison(data_n, X, Y, Z, K, vectors_comparison)

74.2 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
